# Mocked Authorities

#### Use SDX stack for authorities temporarily

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom
export AUTH_INFO_FILE=${WORKING_DIR}/authority_info
echo "" > $AUTH_INFO_FILE
export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info

export AUTH_NETWORK='sharednet1'
export AUTH_NODE_NAME=${USERNAME}-auth
export AUTH_STACK_NAME=${USERNAME}-auth

echo AUTH_NODE_NAME $AUTH_NODE_NAME >> $AUTH_INFO_FILE
echo AUTH_STACK_NAME $AUTH_STACK_NAME >> $AUTH_INFO_FILE

#RIAK Info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo RIAK_PUBLIC_IP: $RIAK_PUBLIC_IP

In [ ]:
#Manually set riak reservation
#echo '8882c0ea-e92a-404e-8637-73b4c38f4101' > AUTH_RESERVATION_FILE

#### Create Lease (if necessary)

In [ ]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation

# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-auth-lease"
echo LEASE_NAME $LEASE_NAME >> $AUTH_INFO_FILE

NODE_MIN=1
NODE_MAX=1

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

In [ ]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

#Poll for node to be ready for ssh
while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_STATUS ${RESERVATION_STATUS}
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $AUTH_INFO_FILE

cat $AUTH_INFO_FILE



#### Start Authority SAFE Server Stack

In [ ]:
RESERVATION_RESOURCE_ID=`cat $AUTH_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
AUTH_NODE_NAME=`cat $AUTH_INFO_FILE | grep AUTH_NODE_NAME | cut -d " " -f 2`
AUTH_STACK_NAME=`cat $AUTH_INFO_FILE | grep AUTH_STACK_NAME | cut -d " " -f 2`

RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

echo Creating Authority SAFE server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template ${WORKING_DIR}/SDX/safe_server.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
        --parameter "network_name=${AUTH_NETWORK}" \
        --parameter "safe_node_name=${AUTH_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${AUTH_STACK_NAME}

In [ ]:
echo Authority server creation complete! 
AUTH_NODE_NAME=`cat $AUTH_INFO_FILE | grep AUTH_NODE_NAME | cut -d " " -f 2`

AUTH_PUBLIC_IP=`openstack server show  --format value -c addresses ${AUTH_NODE_NAME} | cut -d " " -f 2`
echo The Authority public IP is $AUTH_PUBLIC_IP
echo AUTH_PUBLIC_IP $AUTH_PUBLIC_IP >> $AUTH_INFO_FILE

In [ ]:
#AUTH_PUBLIC_IP=`openstack server show --format value -c addresses ${AUTH_NODE_NAME} | cut -d " " -f 2`
AUTH_PUBLIC_IP=`cat $AUTH_INFO_FILE | grep AUTH_PUBLIC_IP | cut -d " " -f 2`
echo AUTH_PUBLIC_IP is $AUTH_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${AUTH_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo Authority node ready for ssh

#### Make delegations to a user
##### Geni delegations, IP delegation, Tag delegation

#### (STEP 3a)

In [ ]:
USERKEYHASH="2JAOVClQyaajkrpe7kxTtqbzW3x__AmwgFfKXyo63Q8="
USERSLICE=${USERNAME}-test
USERIP="192.168.42.1/24"
USERTAG="tag0"

REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

#### (STEP 3b) -- Then go back to "client.ipynb"

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${AUTH_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock auth ${USERKEYHASH} ${USERSLICE} ${USERIP} ${USERTAG} ${AUTH_PUBLIC_IP}"
    
echo Authority delegations done

#### Clean Up Authority

In [ ]:
AUTH_STACK_NAME=`cat $AUTH_INFO_FILE | grep AUTH_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $AUTH_STACK_NAME

LEASE_NAME=`cat $AUTH_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $AUTH_INFO_FILE
mv $AUTH_INFO_FILE ${AUTH_INFO_FILE}.old